In [54]:
! pip install elosports
!pip install sportsdataverse
from elosports.elo import Elo
import sportsdataverse as sdv
import pandas as pd
import datetime

In [55]:
df= sdv.mbb.load_mbb_team_boxscore(seasons=range(2021,2022))
df=df.to_pandas()
def flagging_home(x):
    if x =="home":
      return 1
    else:
      return 0
df['home']=df['team_home_away'].apply(flagging_home)
df.sort_values(by=['game_id','home'],inplace=True)
df.rename(index=df['game_id'],inplace=True)
features=["field_goals_made","field_goals_attempted",
          'three_point_field_goals_made','three_point_field_goals_attempted',
          'free_throws_made','free_throws_attempted',
          'offensive_rebounds','defensive_rebounds',
          'assists','blocks',
          'steals','turnovers',
          'fouls','team_score'
          ]
df.sort_values(by=["game_id","home"],inplace=True)
df.columns

100%|██████████| 1/1 [00:00<00:00,  3.30it/s]


Index(['game_id', 'season', 'season_type', 'game_date', 'game_date_time',
       'team_id', 'team_uid', 'team_slug', 'team_location', 'team_name',
       'team_abbreviation', 'team_display_name', 'team_short_display_name',
       'team_color', 'team_alternate_color', 'team_logo', 'team_home_away',
       'team_score', 'team_winner', 'assists', 'blocks', 'defensive_rebounds',
       'field_goal_pct', 'field_goals_made', 'field_goals_attempted',
       'flagrant_fouls', 'fouls', 'free_throw_pct', 'free_throws_made',
       'free_throws_attempted', 'largest_lead', 'offensive_rebounds', 'steals',
       'team_turnovers', 'technical_fouls', 'three_point_field_goal_pct',
       'three_point_field_goals_made', 'three_point_field_goals_attempted',
       'total_rebounds', 'total_technical_fouls', 'total_turnovers',
       'turnovers', 'opponent_team_id', 'opponent_team_uid',
       'opponent_team_slug', 'opponent_team_location', 'opponent_team_name',
       'opponent_team_abbreviation', 'oppon

In [56]:
df[:6000].shape

(6000, 55)

In [57]:
allTeams 	= set(df['team_name'])
eloLeague 	= Elo(k = 20)

for team in allTeams:
	eloLeague.addPlayer(team)


for idx in range(1,df[:6000].shape[0],2):
	game=df.iloc[idx]
	if game['team_score'] > game["opponent_team_score"]:
		eloLeague.gameOver(winner = game["team_name"], loser = game["opponent_team_name"],winnerHome=True)
	else:
		eloLeague.gameOver(winner = game["opponent_team_name"], loser = game["team_name"],winnerHome=False)

for team in eloLeague.ratingDict.keys():
	print (team, eloLeague.ratingDict[team])

Fighting Bees 1485.1770679032754
Blue Demons 1500
Aggies 1540.2690386241163
Texans 1492.1825829451514
Ramblin' Rams 1492.5814493427204
Pioneers 1372.165008090955
Bearcats 1425.824828937876
Hatters 1463.6436122521905
Wolfpack 1522.5911695518305
Skyhawks 1444.028282895665
Vandals 1359.0518161261853
Sooners 1547.787812484513
Pilots 1446.274833217167
Eutectics 1494.0965179702703
Huskies 1370.6445688824795
Leopards 1510.600776442344
Jackrabbits 1556.2918320173662
Gamecocks 1513.7453993648433
Toreros 1477.9187007590656
Hurricanes 1430.8710936746063
Bulldogs 1552.4981715561516
Blue Hose 1466.7912102458079
Flyers 1532.4481895091487
Tar Heels 1537.1245851990668
Spiders 1534.402662234829
Battling Bishops 1470.9826626513066
Red Raiders 1555.2649191908138
Hawks 1470.8646180133137
Panthers 1474.5083728807067
Ragin' Cajuns 1534.5817273095736
Demons 1468.0901002551673
Big Blues 1500
Colonels 1642.7152886149963
Beach 1452.3834672364226
Hillcats 1490.4399363068057
Bison 1483.4492899281986
Shockers 1565

In [64]:
result=[]
true=[]
for idx in range(6000,df.shape[0],2):
    game=df.iloc[idx]
    result.append(eloLeague.expectResult(eloLeague.ratingDict[game["team_name"]]+100, eloLeague.ratingDict[game["opponent_team_name"]]))
    if game['team_score'] > game["opponent_team_score"]:
        eloLeague.gameOver(winner = game["team_name"], loser = game["opponent_team_name"],winnerHome=True)
        true.append(1)
    else:
        eloLeague.gameOver(winner = game["opponent_team_name"], loser = game["team_name"],winnerHome=False)
        true.append(0)

In [65]:
result=[1 if i>0.5 else 0 for i in result]

In [71]:
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay,f1_score,accuracy_score
pd.DataFrame(confusion_matrix(true, result, normalize='true'),
index=['True 0', 'True 1'],
columns=['Predicted 0', 'Predicted 1'])


,Predicted 0,Predicted 1
True 0,0.176263,0.823737
True 1,0.113426,0.886574


In [72]:
accuracy_score(true,result)

0.4154325798908807